In [4]:
#!pip3 install 'ray[rllib]'

In [5]:
import torch 
from torch import nn

import ray
from ray.rllib.agents import ppo
from ray.rllib.models import ModelCatalog
from ray.rllib.models.torch.torch_modelv2 import TorchModelV2

In [6]:
from models import VisualEncoder

In [7]:
class CustomTorchModel(TorchModelV2, nn.Module):
    def __init__(self, obs_space, action_space, num_outputs, model_config, name): 
        TorchModelV2.__init__(self, obs_space, action_space, num_outputs, model_config, name)
        nn.Module.__init__(self)
        features_dim = 64
        self.visual_encoder = nn.Sequential(
            nn.Linear(obs_space.shape[0], features_dim),
            nn.ReLU(),
            nn.Linear(features_dim, features_dim),
            nn.ReLU(),
        )
        #VisualEncoder(features_dim)
        self.policy_head = nn.Linear(features_dim, action_space.shape[0])
        self.value_head = nn.Linear(features_dim, 1)
        self.value_out = None
        
    def forward(self, input_dict, state, seq_lens):
        #print(input_dict)
        obs = input_dict['obs_flat']
        features = self.visual_encoder(obs)
        action = self.policy_head(features)
        self.value_out = self.value_head(features)
        return action, state
        
    def value_function(self):
        return self.value_out

In [8]:
ModelCatalog.register_custom_model("my_torch_model", CustomTorchModel)

In [12]:
#ray.init()
trainer = ppo.PPOTrainer(env="CartPole-v0", config={
    "framework": "torch",
    "model": {
        "custom_model": "my_torch_model",
        # Extra kwargs to be passed to your model's c'tor.
        "custom_model_config": {},
    },
})

RayActorError: The actor died because of an error raised in its creation task, [36mray::RolloutWorker.__init__()[39m (pid=270, ip=192.168.1.96)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 584, in __init__
    seed=seed)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1376, in _build_policy_map
    conf, merged_conf)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/policy/policy_map.py", line 137, in create_policy
    merged_config)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/policy/policy_template.py", line 247, in __init__
    framework=framework)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/models/catalog.py", line 539, in get_model_v2
    **customized_model_kwargs)
  File "/tmp/ipykernel_25/1936625470.py", line 13, in __init__
IndexError: tuple index out of range

(pid=270) 2021-09-06 17:46:20,485	ERROR worker.py:428 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::RolloutWorker.__init__() (pid=270, ip=192.168.1.96)
(pid=270)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 584, in __init__
(pid=270)     seed=seed)
(pid=270)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/evaluation/rollout_worker.py", line 1376, in _build_policy_map
(pid=270)     conf, merged_conf)
(pid=270)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/policy/policy_map.py", line 137, in create_policy
(pid=270)     merged_config)
(pid=270)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/policy/policy_template.py", line 247, in __init__
(pid=270)     framework=framework)
(pid=270)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/models/catalog.py", line 539, in get_mo

In [ ]:
docker run --ipc

In [10]:
from ray import tune
from ray.rllib.agents.dqn import DQNTrainer
tune.run(DQNTrainer, config={"env": "CartPole-v0"})

Trial name,status,loc
DQN_CartPole-v0_91116_00000,PENDING,


2021-09-06 14:56:12,435	ERROR trial_runner.py:773 -- Trial DQN_CartPole-v0_91116_00000: Error processing event.
Traceback (most recent call last):
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/tune/trial_runner.py", line 739, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/tune/ray_trial_executor.py", line 746, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/_private/client_mode_hook.py", line 82, in wrapper
    return func(*args, **kwargs)
  File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/worker.py", line 1623, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=2780, ip=192.168.1.96)
AttributeError: 'NoneType' object has no attribute 'config'

During handling of the above 

Result for DQN_CartPole-v0_91116_00000:
  {}
  


Trial name,status,loc
DQN_CartPole-v0_91116_00000,ERROR,
Trial name,# failures,error file
DQN_CartPole-v0_91116_00000,1,/root/ray_results/DQN_2021-09-06_14-56-07/DQN_CartPole-v0_91116_00000_0_2021-09-06_14-56-08/error.txt


(pid=2780) 2021-09-06 14:56:12,405	INFO dqn.py:188 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
(pid=2780) 2021-09-06 14:56:12,405	INFO trainer.py:728 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=2780) 2021-09-06 14:56:12,433	ERROR worker.py:428 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::DQN.__init__() (pid=2780, ip=192.168.1.96)
(pid=2780) AttributeError: 'NoneType' object has no attribute 'config'
(pid=2780) 
(pid=2780) During handling of the above exception, another exception occurred:
(pid=2780) 
(pid=2780) ray::DQN.__init__() (pid=2780, ip=192.168.1.96)
(pid=2780)   File "/root/miniconda/envs/py37/lib/python3.7/site-packages/ray/rllib/agents/trainer_template.py", line 136, in __init__
(pid=2780)     Trainer.__init__(self, config, 

TuneError: ('Trials did not complete', [DQN_CartPole-v0_91116_00000])

In [ ]:
from ray.rllib.agents.impala import ImpalaTrainer